In [256]:
import math
import pandas as pd
from itertools import chain, combinations

In [257]:
def euklides_measure(vec1, vec2):
    result = 0
    # ewentualnie mozna liczyc odleglosci euklidesowe dla wektorow o roznej odl
    if len(vec1) != len(vec2):
        return 0
    for i in range(len(vec1)):
        result = result + (vec1[i]-vec2[i])**2
    result = math.sqrt(result)

    #korekta, gdy odleglosc to 0, zapisujemy 0 w inny sposób
    if result == 0:
        result = 1e-21

    return result

In [258]:
def get_csv(x):

    df = pd.read_csv(x)
    leng = len(df.columns)
    return df, leng

In [259]:
def write_to_sim_vec(sim_vec, index, emeasure, vec_of_vecs):
    sim_vec[index] = emeasure
    vec_of_vecs.append(sim_vec)

    return vec_of_vecs


In [260]:
def compare(test, which_line, agent, a_index, dec_atr):
    test = test_file.iloc[which_line]

    #zliczac do zmiennej liczbe wartosci atrybutu decyzyjnego -> podawana w zmiennej
    vec_of_vecs = []

    train_vec = [None] * (a_index - 1)
    test_vec = [None] * (a_index - 1)

    #tworzenie wektora odległości bezwzględnej
    for index, row in agent.iterrows():
        sim_vec = [0] * len(dec_atr)

        for i in range (len(train_vec)):
            train_vec[i] = row[agent.columns[i]]
            test_vec[i] = test[agent.columns[i]]

        emeasure = euklides_measure(train_vec,test_vec)

        #w innej funkcji wybierac wszystkie odpowiadajace dla wybranego k

        for atr in dec_atr:

            if row[agent.columns[a_index-1]] == atr:
                vec_of_vecs = write_to_sim_vec(sim_vec, dec_atr.index(atr), emeasure, vec_of_vecs)


    return vec_of_vecs


In [261]:
def find_dec_atr(files):
    dec_atr = []
    for file in files:
        [agent, a_index] = get_csv(file)
        for index, row in agent.iterrows():
            dec = row[agent.columns[a_index-1]]
            if dec not in dec_atr:
                dec_atr.append(dec)

    return dec_atr

In [262]:
def make_vector(files, test_file, which_line, num_of_dec):
    all_vecs = []
    for file in files:
        [agent, a_index] = get_csv(file)
        all_vecs.append(compare(test_file, which_line, agent, a_index, num_of_dec))
    return all_vecs



In [263]:
def find_the_closest(k, list_v, dec_atr, agent_num):

    final_vec = [[[0 for x in range(len(dec_atr))] for x in range(k)] for x in range(agent_num)]
    ixs_to_delete = [[0 for x in range(len(dec_atr))] for x in range(agent_num)]

    save = False

    #[[[0,0,0..],[..],[..],..]],[[0,0,0..],[..],[..],..]],[[0,0,0..],[..],[..],..]],..]
    for i in range(k):
        for j in range(len(list_v)):
            for l in range(len(list_v[j])):
                for m in range(len(list_v[j][l])):
                    if list_v[j][l][m] != 0:
                        final_vec, save = try_val(final_vec, j, i, list_v[j][l], m)
                        if save:
                            ixs_to_delete[j][m] = l
    # deletion:
    # sorting the inner list to avoid moving important index down

        for i in range(len(ixs_to_delete)):
            for j in sorted(ixs_to_delete[i], reverse= True):
                del list_v[i][j]

    # jesli k-ty obiekt ma taka sama odl jak inne obiekty, to wszystkie obiekty sa brane pod uwage
    # 0.1, 0.2, 0.2, 0.3 -> dla k = 2 bierzemy 3 pierwsze obiekty, bo mają tą samą wartość

    return final_vec

In [264]:
def try_val(final_vec, agent_in_final, k_in_final, vec, m):

    fin = final_vec[agent_in_final][k_in_final]
    save = False

    for i in range(len(fin)):

        if i == m:
            if fin[m] == 0:
                fin[m] = vec[m]
                save = True

            elif fin[m] > vec[m]:
                fin[m] = vec[m]
                save = True

    return final_vec, save


In [265]:
def change_to_relative(k_closest):
    for k in k_closest:
        for l in k:
            sum = 0
            #zliczamy sume
            for m in l:
                sum += m
            #obliczamy wartosci wzgledem sumy i odejmujemy
            # od 1 aby nie byly najmniejsze, tylko najwieksze, aby utworzyc podobienstwo
            for i in range(len(l)):
                l[i] = 1 - l[i]/sum
            #podobniestwo wzgledne (udzial procentowy)
            sum = 0
            for m in l:
                sum += m
            for i in range(len(l)):
                l[i] = l[i]/sum

    # to na koncu (normalizacja)

    return k_closest

In [266]:
def unify_k_vecs(relative_vecs, agent_num):
    unified_vec = [[0 for x in range(len(relative_vecs[0][0]))]for x in range(agent_num)]
    for i in range(len(relative_vecs)):
        for j in range(len(relative_vecs[i])):

            for k in range(len(relative_vecs[i][j])):
                unified_vec[i][k] += relative_vecs[i][j][k]

        for k in range(len(relative_vecs[i][j])):
            unified_vec[i][k] /= len(relative_vecs[i])

    # tu nie normalizować, tylko raz na końcu

    return unified_vec


In [267]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [268]:
#q od 0 do 1 - wartosc procentowa, po osiagnieciu ktorej decyzja jest przelosowana
def Shapley_Shubik(vec, q, dec_atr):

    force_list = [0 for x in range(len(vec))]
    sums = [0 for x in range(len(dec_atr))]
    #na początku szukamy list dla poszczególnych decyzji, zapisujemy ją (listę) jako temp_vec
    for i in range(len(dec_atr)):

        #temp_vec jest pomocniczy, aby obliczyc ostateczną siłę agenta: [dec1a1, dec1a2, dec1a3..]
        temp_vec = [0 for x in range(len(vec))]

        #uzupelnienie wektora
        for m in range(len(vec)):
            temp_vec[m] = vec[m][i]

        #wyciągamy pierszy element tabeli i później odkładamy go na górę (koniec)
        agent_forces = []
        for l in range(len(temp_vec)):

            # print(i)
            # print(len(temp_vec))
            # print(temp_vec)

            x = temp_vec.pop(0)
            forces= []
            subsets = list(powerset(temp_vec))
            sum_to_Shapley = 0
            div = 0

            # print(x)
            # print(temp_vec)
            # print(len(subsets))

            #liczenie wartości sił agentów dla poszczególnych decyzji
            for sub in subsets:
                if sub:
                    sum = 0
                    for s in sub:
                        sum += s
                    if sum > q:
                        pass
                    elif sum + x > q:
                        sum_to_Shapley += (math.factorial(len(sub)) * math.factorial((len(temp_vec)+1)-len(sub)-1))

                else:
                    if x > q:
                        sum_to_Shapley += (math.factorial(len(sub)) * math.factorial((len(temp_vec)+1)-len(sub)-1))

            temp_vec.append(x)
            agent_forces.append(sum_to_Shapley/math.factorial(len(temp_vec)))
        sums[i] = agent_forces

            #liczenie sumy wartości sił jako siły agenta
    final_indexes = [0 for x in range(len(vec))]
    for i in range(len(sums)):
        for j in range(len(sums[i])):
            final_indexes[j] += sums[i][j]

            #liczenie sił poparcia dla wszystkich decyzji



            #liczenie ostatecznej siły
    return force_list

    # algorytm poprawić bo musi być po agentach a nie po decyzjach!!!


In [269]:
def count_final_vec(unified_vec, f_list):

    help_vec = [[0 for x in range(len(unified_vec[0]))]for x in range(len(unified_vec))]
    final_vec = [0 for x in range(len(unified_vec[0]))]

    for i in range(len(unified_vec)):
        for j in range(len(unified_vec[i])):
            help_vec[i][j] = unified_vec[i][j] * f_list[i][j]

    for i in range(len(help_vec)):
        for j in range(len(help_vec[i])):
            final_vec[j] += help_vec[i][j]

    return final_vec

In [270]:
files = ['d1.csv', 'd2.csv', 'd3.csv', 'd4.csv', 'd5.csv', 'd6.csv', 'd7.csv']
test_file, t_index = get_csv('test_w.csv')
dec_atr = find_dec_atr(files)
ageent_num = len(files)

# for i in range(len(test_file)):

line_to_compare = 0

list_v = make_vector(files, test_file, line_to_compare, dec_atr)

k_closest = find_the_closest(3, list_v, dec_atr, ageent_num)

# print("k_closest: ", k_closest)

relative_vecs = change_to_relative(k_closest)

# print("relative_vecs: ",relative_vecs)

unified_vec = unify_k_vecs(relative_vecs, ageent_num)

# print("unified_vec: ",unified_vec)

f_list = Shapley_Shubik(unified_vec, 0.5, dec_atr)

# print("f_list: ",f_list)

# final_vec = count_final_vec(unified_vec,f_list)
#
# print("final_vec: ",final_vec)

# print('decyzja agentow: ', dec_atr[final_vec.index(max(final_vec))])
# print('decyzja poprawna: ', test_file.iloc[line_to_compare][t_index - 1])
# print('Czy dobra decyzja?: ', dec_atr[final_vec.index(max(final_vec))] == test_file.iloc[line_to_compare][t_index - 1], end='\n\n')

    # 160/94  -> T/F
    # do poprawy unifikacja agentów -> nie robić jej
    # do dodania składowanie wyników w jakimś słowniku..
    # siła ma być wartością a nie wektorem (dodać wartości sił)

[[0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.20952380952380953, 0.20952380952380953, 0.0761904761904762, 0.0761904761904762], [0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285], [0.12380952380952381, 0.12380952380952381, 0.07380952380952381, 0.04047619047619048, 0.04047619047619048, 0.2571428571428571, 0.3404761904761905], [0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]]
[0.5523809523809524, 0.5523809523809524, 0.5023809523809524, 0.5357142857142857, 0.5357142857142857, 0.6190476190476191, 0.7023809523809523]
